# Summarize the QMCPack Output Data
The data published in the MDF are the raw outputs of simulation codes. This notebook processes it into a dataframe of energies following the instructions in the [README](https://data.materialsdatafacility.org/mdf_open/qmc_ml_v1.1/README.md)

In [1]:
from concurrent.futures import ThreadPoolExecutor
from tempfile import TemporaryDirectory
from foundry import Foundry
from openbabel import pybel
from threading import Lock
from subprocess import run
from pathlib import Path
from time import sleep
from math import isclose
from tqdm import tqdm
import pandas as pd
import requests
import re

In [2]:
pybel.ob.obErrorLog.SetOutputLevel(0)

Configuration

In [3]:
qmca_path = '/home/lward/Software/qmcpack-3.15.0/nexus/bin/qmca'  # Path to the QMCA executable on your system

In [4]:
num_fragments = 1175  # Number of fragments
starting_theories = ['HF', 'PBE', 'PBE0', 'B3LYP']  # Theories used for the starting wfc

In [5]:
dataset_path = '/mdf_open/qmc_ml_v1.1'  # Path to the dataset on the MDF services
base_url = f'https://data.materialsdatafacility.org/{dataset_path}'  # HTTP access
endpoint = '82f1b5c6-6e9b-11e5-ba47-22000b92c6ec'  # Globus endpoint

## Define the Key Functions
We need something to iterate through the dataset and process the data once downloaded

### Download Functions
The directories of this dataset are laid out in predictable ways. The path to the DMC computation is always `/frag_{frag_id}/{base_wfc}/DMC/` and filenames are also well-defined.

We create functions for accessing specific data needed in our datasets

Make a Foundry client so we can use it to access Globus APIs

In [6]:
foundry = Foundry()  # We'll use this to help you access the Globus API

Make a requests session with retries to make HTTP more robust

In [7]:
s = requests.Session()
s.mount(f'https://{base_url}/', requests.adapters.HTTPAdapter(max_retries=8, pool_block=True))
def make_request(url):
    while True:
        try:
            return s.get(url, timeout=30)
        except (requests.Timeout, requests.ConnectionError):
            sleep(5)

In [8]:
def make_frag_name(frag_id: int) -> str:
    """Format the name of a fragment appropriately
    
    Args:
        frag_id: ID of the fragment
    Returns:
        Fragment name
    """
    return f'frag_{frag_id:05d}'
assert make_frag_name(1) == 'frag_00001'

In [9]:
def get_xyz(frag_id: int) -> str:
    """Get the molecular geometry for this computation
    
    Returns:
        Geometry in XYZ format
    """
    
    frag_name = make_frag_name(frag_id)
    url = f'{base_url}/{frag_name}/{frag_name}.xyz'
    xyz = make_request(url).text
    
    # Add in the header information to make it easier to parse
    n_atoms = xyz.count('\n')
    return f'{n_atoms}\n{frag_name}\n{xyz}'
xyz = get_xyz(1)
assert xyz.startswith('5\nfrag_00001\n  C   0.95761131890095')

In [10]:
def get_base_theory_output(frag_id: int, theory: str) -> str:
    """Get the PySCF output file from the base level of theory
    
    
    Args:
        frag_id: ID of the fragment
        theory: Theory used to compute the energy
    Returns:
        Output file in plaintext
    """
    
    frag_name = make_frag_name(frag_id)
    url = f'{base_url}/{frag_name}/{theory}/frag{frag_id:05d}.out'
    return make_request(url).text
b3lyp_out = get_base_theory_output(1, 'B3LYP')
assert b3lyp_out.startswith('#INFO:')

In [11]:
lock = Lock()  # Only one Globus query per process
def get_dmc_output(frag_id: int, theory: str) -> str:
    """Get the scalar.out file for the last DMC series
    
    
    Args:
        frag_id: ID of the fragment
        theory: Theory used to compute the initial guess
    Returns:
        Scalar file in plaintext
    """
    
    # Determine the filename of the output 
    frag_name = make_frag_name(frag_id)
    dmc_path = f'{dataset_path}/{frag_name}/{theory}/DMC/'
    output_file = None
    with lock:
        for entry in foundry.transfer_client.operation_ls(endpoint, dmc_path):
            if entry['name'].endswith('s015.scalar.dat'):  # Find the last QMC step
                output_file = entry['name']
                break
    assert output_file is not None
    
    url = f'{base_url}/{frag_name}/{theory}/DMC/{output_file}'
    result = make_request(url)
    assert result.status_code == 200, result.text
    return result.text
dmc_out = get_dmc_output(1, 'B3LYP')
assert dmc_out.startswith('#   index')

### Processing Functions
Functions that take the downloaded information and render useable information out of it

In [12]:
def get_molecule_identifiers(xyz: str) -> (str, str):
    """Get SMILES string from the XYZ format
    
    Args:
        xyz: Structure of molecule
    Returns:
        - SMILES string
        - InChI string
    """
    mol = pybel.readstring('xyz', xyz)
    return mol.write('smi').split("\t")[0], mol.write('inchi').strip()
smiles, inchi = get_molecule_identifiers(xyz)

In [13]:
energy_re = re.compile(r'converged SCF energy = (?P<energy>[-\d\.]+)')

def get_base_energy(pyscf_out: str) -> float:
    """Get the PySCF energy
    
    Args:
        pyscf_out: Output from a PySCF computation
    Returns:
        Energy
    """
    
    matches = energy_re.findall(pyscf_out)
    if len(matches) == 0:
        raise ValueError('No SCF energy!?')
    return float(matches[0])
assert isclose(get_base_energy(b3lyp_out), -40.50228077)

In [14]:
qmca_re = re.compile(r'series 0 +(?P<mean>[-\.\d]+) \+/- (?P<err>[-\.\d]+)')
def get_dmc_energy(scalar_out: str) -> (float, float):
    """Get the output energy and variance from a QMC computation
    
    Calls QMCA to determine them automatically
    
    Args:
        scalar_out: Scalar output from the simulation
    Returns:
        Mean and error for the computation
    """

    # Invoke QMCA
    with TemporaryDirectory(dir='.') as tmp:
        # Temporary directory to avoid conflicts between threads
        scalar_path = Path(tmp) / 'test.g000.s000.scalar.dat'
        with open(scalar_path, 'w') as fp:
            print(scalar_out, file=fp)
        output = run([qmca_path, '-q', 'ev', scalar_path], capture_output=True, text=True)
    
    # Get the energies
    match = qmca_re.search(output.stdout)
    if match is None:
        raise ValueError(output)
    energy, error = match.groups()
    return float(energy), float(error)
energy, error = get_dmc_energy(dmc_out)
assert isclose(energy, -40.507445, abs_tol=error)
assert isclose(error, 0.000268, abs_tol=1e-4)

## Run the full thing
Get the data for every fragment

In [15]:
def get_record(frag_id: int) -> dict:
    """Get all the information for a certain fragment
    
    Args:
        frag_id: ID number of the fragment
    Returns:
        Dictionairy containing a complete record of the fragment
    """
    # Get the XYZ file
    xyz = get_xyz(frag_id)
    
    # Convert it to SMILES then start the record
    smiles, inchi = get_molecule_identifiers(xyz)
    record = {
        'fragment': frag_id,
        'smiles': smiles,
        'inchi': inchi,
        'xyz': xyz,
    }
    
    # Gather data from each theory
    for theory in starting_theories:
        # Get the starting energy for this level
        output = get_base_theory_output(frag_id, theory)
        record[theory] = get_base_energy(output)
        
        # Get the DMC energy
        scalar_out = get_dmc_output(frag_id, theory)
        energy, error = get_dmc_energy(scalar_out)
        record[f'DMC({theory})'] = energy
        record[f'DMC({theory})_err'] = error

    return record
get_record(num_fragments)

{'fragment': 1175,
 'smiles': 'o1cnnc1',
 'inchi': 'InChI=1S/C2H2N2O/c1-3-4-2-5-1/h1-2H',
 'xyz': '7\nfrag_01175\n  O   -0.35718091715119     -1.10236409490513     -0.00399495556771\n  C   -1.09840997106521      0.03544868082879     -0.00207290982889\n  N   -0.38702425410702      1.10402513231628      0.00033442532297\n  N   0.94836418591065      0.67604166909837      0.00003461309664\n  C   0.90622109656799     -0.60686429057309     -0.00224705301157\n  H   -2.17145828261521     -0.05765530136078     -0.00275374497995\n  H   1.72308814235999     -1.30793179540444     -0.00320037513148\n',
 'HF': -260.704596557406,
 'DMC(HF)': -262.02708,
 'DMC(HF)_err': 0.000671,
 'PBE': -261.904890823716,
 'DMC(PBE)': -262.036121,
 'DMC(PBE)_err': 0.000629,
 'PBE0': -261.89986731426,
 'DMC(PBE0)': -262.03627,
 'DMC(PBE0)_err': 0.000607,
 'B3LYP': -262.056860579202,
 'DMC(B3LYP)': -262.036063,
 'DMC(B3LYP)_err': 0.000589}

In [16]:
frag_iter = range(1, num_fragments + 1)
with ThreadPoolExecutor(16) as ex:
    records = [r for r in tqdm(ex.map(get_record, frag_iter), total=num_fragments)]
records = pd.DataFrame(records)
records.head(5)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1175/1175 [1:19:21<00:00,  4.05s/it]


,fragment,smiles,inchi,xyz,HF,DMC(HF),DMC(HF)_err,PBE,DMC(PBE),DMC(PBE)_err,PBE0,DMC(PBE0),DMC(PBE0)_err,B3LYP,DMC(B3LYP),DMC(B3LYP)_err
0,1,C,InChI=1S/CH4/h1H4,5\nfrag_00001\n C 0.95761131890095 -0.0...,-40.213319,-40.505945,0.000254,-40.463311,-40.507007,0.000324,-40.474441,-40.506365,0.000276,-40.502281,-40.507445,0.000268
1,2,N,InChI=1S/H3N/h1H3,4\nfrag_00002\n N 0.95838313325616 0.0...,-56.217747,-56.548449,0.000294,-56.506473,-56.549669,0.000311,-56.512004,-56.549934,0.000333,-56.548145,-56.548902,0.001752
2,3,O,InChI=1S/H2O/h1H2,3\nfrag_00003\n O 1.01914638660453 -0.0...,-76.056712,-76.418847,0.000558,-76.372988,-76.420451,0.000307,-76.374332,-76.420688,0.000326,-76.422727,-76.414041,0.001772
3,4,C#C,InChI=1S/C2H2/c1-2/h1-2H,4\nfrag_00004\n C 0.98989790782649 0.0...,-76.849752,-77.311674,0.000345,-77.249870,-77.313619,0.000428,-77.254838,-77.313950,0.000327,-77.312373,-77.313163,0.000452
4,5,C#N,InChI=1S/CHN/c1-2/h1H,3\nfrag_00005\n C 0.97091888623401 -0.0...,-92.908793,-93.400895,0.000346,-93.343723,-93.404958,0.000470,-93.342293,-93.403561,0.000381,-93.409677,-93.402739,0.000344


Save it to disk in CSV format

In [17]:
records.to_csv('QMC_AMIONS_NI_LE.csv', index=False)